### Ingest drivers.json File

Step 1 - Read the Json file using the spark datafame reader API

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
                                 ])

In [0]:
driver_schema = StructType(fields=[StructField("driverId", IntegerType(), True),
                                 StructField("driverRef", StringType(), True),
                                 StructField("number", IntegerType(), True),
                                 StructField("code", StringType(), True),
                                 StructField("name", name_schema, True),
                                 StructField("dob", DateType(), True),
                                 StructField("nationality", StringType(), True),
                                 StructField("url", StringType(), True)
                                 ])

In [0]:
drivers_df = spark.read.schema(driver_schema).json("/mnt/f1dlportfolio/raw/drivers.json")

In [0]:
drivers_df.printSchema()

Step 2 - Rename columns and add new column

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id").withColumnRenamed("driverRef", "driver_ref") \
                        .withColumn("ingestion_date", current_timestamp()) \
                        .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

Step 3  - Drop unwanted columns from the dataframe

In [0]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

Step 4 Write output the processed container in parquet file

In [0]:
drivers_final_df.write.mode("overwrite").parquet("/mnt/f1dlportfolio/processed/drivers.parquet")

In [0]:
dbutils.notebook.exit("Success")